imprements by https://www.kaggle.com/code/awsaf49/birdclef23-effnet-fsr-cutmixup-train

In [1]:
import sys
sys.path.append('/kaggle/input/birdclef-2023-baseline')
sys.path.append('/kaggle/input/torchlibrosa-master')
sys.path.append('/kaggle/input/tinytag-master')

# Impoert library

In [2]:
import os
import random
from glob import glob
from pathlib import Path
import yaml
from tqdm import tqdm
import IPython.display as ipd

import numpy as np
import pandas as pd

import matplotlib as mpl
cmap = mpl.cm.get_cmap('coolwarm')
import matplotlib.pyplot as plt

import librosa
import wandb
from sklearn import metrics

import torch
from torch import nn
from torch.nn import functional as F
from torch.cuda.amp import autocast, GradScaler

from timm.scheduler import CosineLRScheduler

In [3]:
from config import CFG
from pytorch_model import BirdCLEF23Net
import pytorch_modeler as modeler
import pytorch_preprocessing as prep
import common as com

In [4]:
modeler.set_seed(CFG.seed)
# setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print('Debug :', CFG.debug)

cuda:0
Debug : False


In [5]:
from datetime import datetime
import pytz

# 日本時間のタイムゾーンを設定
jst = pytz.timezone('Asia/Tokyo')

# 現在時刻を取得し、日本時間に変換
now = datetime.now(jst)

# 現在時刻を文字列に変換
now_str = now.strftime('%Y-%m-%d %H:%M:%S')

print(now_str)

2023-05-04 22:35:02


# Data Frame

In [6]:
test_paths = glob('/kaggle/input/birdclef-2023/test_soundscapes/*ogg')
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df['filename'] = test_df.filepath.map(lambda x: x.split('/')[-1].replace('.ogg',''))
test_df.head(2)

,filepath,filename
0,/kaggle/input/birdclef-2023/test_soundscapes/s...,soundscape_29201


# Inference

In [7]:
test_ds = prep.BirdDataset(test_df, is_train=False, is_test=True)

test_dataloader = torch.utils.data.DataLoader(
    dataset=test_ds,
    batch_size=CFG.test_bs,
    shuffle=False,
    num_workers = os.cpu_count(),
    collate_fn = modeler.inference_collate,
    )

In [8]:
inference_model_path='/kaggle/working/multi_label/exp5_attblock/fold-0.pth'

In [9]:
num_classes = CFG.num_classes
   
# Compute batch size and number of samples to drop
infer_bs = CFG.test_bs
model = BirdCLEF23Net(num_classes=CFG.num_classes, pretrained=False)
    
# Test
print('# Loading model')
model.load_state_dict(torch.load(inference_model_path, map_location=device), strict=True)
model.to(device)

# Predict on the validation data for oof result
print('# Infering Test')
model.eval()
test_pred = []
row_id = []
with torch.no_grad():
    for inputs, sample_info in tqdm(test_dataloader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        outputs = outputs['clipwise_output']
        test_pred.append(outputs.detach().cpu().numpy())
        row_id.extend(sample_info['row_id'])

test_pred = np.concatenate(test_pred, axis=0)

# Loading model
# Infering Test


  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([120, 1, 224, 196])


100%|██████████| 1/1 [00:06<00:00,  6.56s/it]


In [10]:
bird_names = CFG.class_names
sub_cols = ['row_id'] + bird_names
sub_df = pd.DataFrame(columns=sub_cols)
sub_df['row_id'] = row_id
sub_df[bird_names] = test_pred

In [11]:
sub_df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.027955,0.013533,0.027834,0.008693,0.008465,0.016397,0.009202,0.006020,0.064060,...,0.004706,0.003341,0.010490,0.007063,0.013559,0.009365,0.013433,0.005802,0.012018,0.027144
1,soundscape_29201_10,0.023781,0.036632,0.024304,0.019829,0.013255,0.017344,0.018369,0.022532,0.026145,...,0.011760,0.015976,0.034352,0.021227,0.032004,0.026697,0.019251,0.017176,0.019280,0.026659
2,soundscape_29201_15,0.018125,0.031463,0.017898,0.013299,0.006059,0.015814,0.011503,0.005935,0.029164,...,0.007297,0.009288,0.051044,0.012239,0.022385,0.007955,0.007467,0.012317,0.012668,0.012654
3,soundscape_29201_20,0.016734,0.013679,0.023653,0.015441,0.011324,0.012141,0.009352,0.008399,0.022716,...,0.005093,0.017389,0.028923,0.019496,0.006262,0.005610,0.010177,0.004712,0.003744,0.007810
4,soundscape_29201_25,0.019327,0.016659,0.025294,0.019227,0.011236,0.012380,0.013317,0.013991,0.032197,...,0.007514,0.013588,0.030656,0.016274,0.023139,0.010955,0.008763,0.012210,0.010178,0.011182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,soundscape_29201_580,0.016111,0.016575,0.009291,0.007226,0.006875,0.006779,0.015560,0.010114,0.027388,...,0.003777,0.016338,0.015993,0.004986,0.025644,0.007070,0.007002,0.010176,0.004072,0.004006
116,soundscape_29201_585,0.044478,0.066410,0.012447,0.015839,0.018727,0.020567,0.075392,0.010605,0.016540,...,0.020410,0.015275,0.027965,0.011935,0.040307,0.026051,0.037448,0.025083,0.012366,0.012429
117,soundscape_29201_590,0.031037,0.053105,0.023680,0.022339,0.014140,0.009066,0.051842,0.013280,0.022310,...,0.014960,0.018679,0.051503,0.014692,0.052931,0.017702,0.026921,0.021144,0.015532,0.008587
118,soundscape_29201_595,0.059088,0.045016,0.055907,0.027591,0.064755,0.020847,0.023889,0.037309,0.060771,...,0.011497,0.025758,0.048454,0.040374,0.029621,0.018889,0.029564,0.011192,0.009838,0.014603


In [12]:
sub_df.to_csv('submission.csv',index=False)